In [ ]:
#!pip install transformers==4.24

In [ ]:
#!pip install sentencepiece

In [1]:
# import
import re
import json
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, random_split
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel


/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset

In [2]:
# Dataset class
class BeautyDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        # define variables    
        self.input_ids = []
        self.attn_masks = []
        #self.labels = []
        
        # iterate through the dataset
        for txt, label in zip(txt_list, label_list):
            txt= txt.replace("\n", "")
            #txt_ = (txt[:256]) if len(txt) > 256 else txt

            # prepare the text
            prep_txt = f'<s> Title: {label} [SEP] Content: {txt}</s>'
            # tokenize
            encodings_dict = tokenizer(prep_txt, truncation=True,
                                       max_length=max_length, padding="max_length")
            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            #self.labels.append(label)

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


In [3]:
def read_content_title():
  import glob

  path = r'/home/khinthethtar/text-generation-research/all' # folder
  all_files = glob.glob(path + "/*.csv")

  list_ = []

  for filename in all_files:
    try:
      #print(filename)
      df_t = pd.read_csv(filename, index_col=None, header=None, skiprows=1, encoding='utf-8')
      list_.append(df_t)
    except:
      print(f"reading error in{filename}")



  frame = pd.concat(list_, axis=0, ignore_index=True)
  print(f"length of data frame: {len(frame)}")
  return frame

In [4]:
# Data load function
def load_beauty_dataset(tokenizer, random_seed = 1):
    # load dataset and sample.
    #df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    df = read_content_title()
    df = df[[0, 1]]
    df.columns = ['content', 'title']
    df = df.sample(37366, random_state=1)
    
    max_length = max([len(tokenizer.encode(description)) for description in df['content']])
    print("Max length: {}".format(max_length))

    dataset = BeautyDataset(df['content'].tolist(), df['title'].tolist(), tokenizer, max_length=512)
    dataset.__getitem__(5)

    
    train_size = int(0.9 * len(dataset))
    train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])
    print(len(dataset))

    # return
    return train_dataset, val_dataset

In [5]:
from transformers import T5Tokenizer, AutoModelForCausalLM, GPT2LMHeadModel
  
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium", bos_token='<s>', eos_token='</s>', pad_token='<pad>')

model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 1024)

In [7]:
for trial_no in range(1):
  print("Loading dataset...")
  train_dataset, val_dataset = load_beauty_dataset(tokenizer, trial_no)

Loading dataset...
reading error in/home/khinthethtar/text-generation-research/all/beauty_796718.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_427210.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_959723.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_410930.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_899707.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_927244.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_667202.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_687498.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_761047.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_464814.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_308484.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_642477.c

reading error in/home/khinthethtar/text-generation-research/all/beauty_833496.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_580047.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_962220.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_484913.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_759824.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_657314.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_776394.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_912263.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_857602.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_541467.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_450957.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_442761.csv
reading error in

reading error in/home/khinthethtar/text-generation-research/all/beauty_892661.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_504636.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_811997.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_997207.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_426099.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_885282.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_742658.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_654533.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_489494.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_866750.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_472777.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_516918.csv
reading error in

reading error in/home/khinthethtar/text-generation-research/all/beauty_927863.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_875494.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_684990.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_711011.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_661482.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_704916.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_838887.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_676631.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_504323.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_698902.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_558784.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_820667.csv
reading error in

reading error in/home/khinthethtar/text-generation-research/all/beauty_914840.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_327371.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_726665.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_658235.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_489743.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_773465.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_790702.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_946696.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_432938.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_872483.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_862710.csv
reading error in/home/khinthethtar/text-generation-research/all/beauty_900883.csv
reading error in

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5.py:220: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated"


37366


In [ ]:
print("Start training...")
training_args = TrainingArguments(output_dir=r'/home/khinthethtar/gpt2_rinna_epoch_5_data_37366', num_train_epochs=5, 
                                logging_steps=10000, load_best_model_at_end=True,
                                save_strategy='steps',
                                evaluation_strategy="steps",
                                save_steps=10000,
                                per_device_train_batch_size=12, per_device_eval_batch_size=12,
                                learning_rate=0.001,
                                warmup_steps=1, weight_decay=0.0001, logging_dir='logs')


trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset,
          eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                                'attention_mask': torch.stack([f[1] for f in data]),
                                                                'labels': torch.stack([f[0] for f in data])})
trainer.train()                                                  

Start training...


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 33629
  Num Epochs = 5
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 14015
  Number of trainable parameters = 336129024


Step,Training Loss,Validation Loss


In [ ]:
import os

output_dir = r'/home/khinthethtar/gpt2_rinna_epoch_5_data_37366/final'

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`

model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Models/title_to_text_generation/gpt2_rinna_epoch_5_data_20000")

In [ ]:
text = '【ブローチェ　アヴェダ】また、ピンクブラウンにはメリットが多いのも魅力の一つ。'
prompt = f' Title: {text}\n Content:'
generated = tokenizer(f"<s> {prompt}", return_tensors="pt").input_ids
sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=512, top_p=0.90, 
            temperature=0, num_return_sequences=1)
pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
           

In [ ]:
print(pred_text)

ref: https://qiita.com/m__k/items/36875fedf8ad1842b729